In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 free semantically annotated corpus that anyone can edit!

The Groningen Meaning Bank (GMB), developed at the University of Groningen, comprises thousands of texts in raw and tokenised format, tags for part of speech, named entities and lexical categories, and discourse representation structures compatible with first-order logic. The current (development) version of the GMB is accessible via the GMB Explorer. The multi-lingual successor of the GMB is the Parallel Meaning Bank.

CoNLL format
https://www.clips.uantwerpen.be/conll2002/ner/
Software and Data

The data consists of two columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word and the second the named entity tag. The tags have the same format as in the chunking task: a B denotes the first item of a phrase and an I any non-initial word. There are four types of phrases: person names (PER), organizations (ORG), locations (LOC) and miscellaneous names (MISC). Here is an example:

        Wolff B-PER
            , O
    currently O
            a O
   journalist O
           in O
    Argentina B-LOC
            , O
       played O
         with O
          Del B-PER
       Bosque I-PER
           in O
          the O
        final O
        years O
           of O
          the O
    seventies O
           in O
         Real B-ORG
       Madrid I-ORG
            . O

The data consists of three files per language: one training file and two test files testa and testb. The first test file will be used in the development phase for finding good parameters for the learning system. The second test file will be used for the final evaluation.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import glob
import re
from tqdm import tqdm
import ast

### Load Data

In [ ]:
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
features.head()

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"#for testing do something
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
train.head()

Extract positions from dataframe location field. 
Stackoverflow:
https://stackoverflow.com/questions/4289331/how-to-extract-numbers-from-a-string-in-python


In [ ]:
import re
def extract_numbers(x):
        sentence = x
        s = [float(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
        return s

#### Transformation of annotations spans from char form to token/word level.
That is, if in the competition the annotations are presented in the form of the location-character-start location-character-stop  After the transformation, the format will be: annotation to token position x (where the position is word or character space in the present implementation)
#### Spacy training data format for Named Entity Recognition (NER):

The format is a list of tuple: text and dictionary of spans.

TRAIN_DATA = [

('Who is Shaka Khan?', {'entities': [(7, 17, 'PERSON')]}),

('I like London and Berlin.', {'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]})

]

Due to the fact that spans are in char level there are some limitation: 
* tokenisation is limited to none :) every character including white space is tokenized. 
* it is not wise to split the sentences befeore geting the span value at token level.

#### Two set of labels : features and cases. 
* Features were coded using the feature number. Feature 0 is coded for exemple : NBME0
* Cases were coded using the case number. Case 0 is coded for exemple : NBME-C-0 . the file with case numbers is used to test the model.(fewer labels -more cases- a better validation split) Considerig thata therea are some features with very few cases, simple validation without CV is challenging. 

#### Why loops instead of pandas ... For clarity. 
It easier for me to see ..

There are zillions better ways to do the conversion includig spacy conll convertor. The Python motto "batteries included" is meant to convey the idea that Python comes with everything you need. But...
A single line break make the diference between a **F1** score of **0.1** and **0.69**

#### What’s the Point?

Newbie perspective: same sentence repeated hundred times with different labels: the most extreme case: one label per text has catrastofic results.

https://stackoverflow.com/questions/67055339/how-to-handle-repeating-text-data-but-with-different-labels-or-classes

What the script do: tag each individual pacient note with spans and labels coreponding to each feature. 

At the end the text look like this:

[['17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING. STARTED 2-3 MO AGO. IT STARTED SUDDENLY. DOES NOT RECALL ANY TRIGGERING EVENTS. IT COMES AND GOES, IT HAPPENED 5-6 TIMES SINCE IT STARTED. IT LASTS 3-4 MIN, AFTER THAN  JUST GOES AWAY. HE HAS ALSO EXPERIENCING SOB, PRESSURE ON HER CHEST WHEN HE HAS THIS ATTACK. HE IS A COLLEGE STUDENT, EXPERIENCING SOME STRESS RECENTLY. \r\nDENIES COUGH, CHEST PAIN.\r\nROS NEG EXCEPT AS ABOVE.\r\nPMH NONE. MEDS ATEROL, FOR HIS STUDIES, SHARING W HIS ROOMMATE. NKDA.\r\nPSH/ HOSP/ TRAVEL/ TRAUMA NONE.\r\nFH MOM HAS THYROID PROBLEMS.\r\nSH SEX ACTIVE W GIRLFRIEND, NO STDS, USING CONDOMS. SMOKE NONE. ETOH ONLY WEEKENDS. DRUG ONLY ONCE, 1 MO AGO.',

  {'entities': [[532, 556, 'NBME1'],
  
    [263, 284, 'NBME2'],
    
    [131, 145, 'NBME3'],
    
    [150, 168, 'NBME3'],
    
    [258, 261, 'NBME7'],
    [32, 46, 'NBME9'],
    
    [48, 66, 'NBME10'],
    
    [0, 6, 'NBME11'],
    
    [7, 8, 'NBME12']]}],
    
#### The tuple are saved in a spacy like format. 
Theoretically the results can be imported in spacy and used for NER labeleing from within spacy. But... there is a lot to clean ... :)

In [ ]:
alpha=0

TRAIN_DATA3 = []
TRAIN_DATA4 = []
dumb_data=0
#for i in range(150):
for i in range(len(patient_notes)):
    #print(i)
    pn_num=patient_notes.pn_num[i]
    text=str(patient_notes.pn_history[i])
    temp_df=pd.DataFrame()
    temp_df=train[train["pn_num"]==pn_num]
    temp_df=temp_df.reset_index()
    temp_df['location_string']=temp_df['location'].apply(lambda x: str(x))
    temp_df['spans']=temp_df['location_string'].apply(lambda x: extract_numbers(x))
    temp_df['span_length'] = temp_df['spans'].apply(lambda x: len(x))
    if len(temp_df)!=0:
        #contor_lista=working_corpus.annotation_length[i]
            ent_dic = {}
            ent_dic2 = {}
            lista1=[]
            lista2=[]
            for j in range(len(temp_df)):
                #print("sunt j",j)
                temp_df.feature_num[j]
                code_is="NBME"+str(temp_df["feature_num"][j])
                code_is2="NBME"+"-C-"+str(temp_df["case_num"][j])
                #extract spans
                spans=[]
                alpha=int((temp_df.span_length[j])/2)
                k=0

                for m in range(alpha):
                        #print('sunt m',m)

                        spans=temp_df["spans"][j]

                        #print('spansx',spans[k])
                        #print('spansy',spans[k+1])
                        #print("lista este",lista1)
                        #stringul=[]
                                #stringul=[]
                        #print(train_df["feature_num"][i])

                        #dict_list.append([spans[k],spans[k+1],code_is])
                        #lista1=[spans[k],spans[k+1],code_is]
                        lista1.append([int(spans[k]),int(spans[k+1]),code_is])
                        lista2.append([int(spans[k]),int(spans[k+1]),code_is2])
                        
                        #print(lista1)
#                         ent_dic = {}
#                         ent_dic2 = {}
                    #     #ent_dic['entities'] = dict_list[-1]
                        ent_dic['entities'] = lista1#single value lista2 multiple values
                        ent_dic2['entities'] = lista2 #single value lista2 multiple values 
                    #     #tuple_list.append(tupla)
#                         TRAIN_DATA3.append([text,ent_dic])
#                         TRAIN_DATA4.append([text,ent_dic2])
                        #print(k)
                #         ent_dic2={}
                #         ent_dic2["entities"] = lista1
                #         #tupla=(text,{"entities"+":", location_is[k],location_is[k+1]})
                #         TRAIN_DATA.append([text,ent_dic2])
                        k=k+2
            TRAIN_DATA3.append([text,ent_dic])
            TRAIN_DATA4.append([text,ent_dic2])
    else:
        dumb_data=dumb_data+1
        #print("An Ai without data is like a flower without water")
print("useless pn:",dumb_data)

In [ ]:
len(TRAIN_DATA3)

In [ ]:
TRAIN_DATA3[1:4]

In [ ]:
#ents

#### Export to conll2003 format:

 Here is an example:

        Wolff 0
            , O
    currently O
    HEART     'NBME5
    POUNDING  'NBME5

            a O
   ====New Line ====
   The elxt is labbles token by token each token on a row with a label associated.
   Sentences are separate by new line. 
   
 The result is the file:conll_corpus_train_lab_patient_notes.conll
 
     Which is the only one important for NBME.
     
     The rest of the notebook represent trial and errors for augumenting the text. 
           

In [ ]:
import spacy

data = TRAIN_DATA3
#data=TRAIN_DATA3[1:5]
nlp = spacy.blank("en")

# Construction from class
# from spacy.pipeline import EntityRuler
# ruler = EntityRuler(nlp, overwrite_ents=True)
#myfile = open('xyz.txt', 'w')
i=0
with open('conll_corpus_train_lab_patient_notes.conll', 'w') as f:
   
    for text, labels in data:
        doc=[]
        #print(text)
        doc = nlp(text)
    
        ents = []
        labels["entities"]=sorted(labels["entities"])
        for start, end, label in labels["entities"]:
            #print(start,end,label)
           
            ents.append(doc.char_span(start, end, label))
        
        #print(ents)
        try:
            doc.ents = ents
            next_step=0
            for tok in doc:
                label = tok.ent_iob_
                if tok.ent_iob_ != "O":
                    label += '-' + tok.ent_type_
                    #comment for adding new line after sentences
                    #print(label)
                    if (label.__contains__("B-")):
                                    f.write('\n')
                                    #print("new line __-")
                                    f.write(f'{tok} {label}\n')
                                    #print(tok,label)
                                    next_step=0
                    else:
                                    f.write(f'{tok} {label}\n')
                                    #print(tok,label)
                                    next_step=1
#                     f.write('\n')
#                     print("new line end of sentence1","   ")
                else:
                    humpty_dumpty=1
            f.write('\n')
            #print("new line end of sentence1","   ")
                
            if next_step==1:
                    for tok in doc:
                        label = tok.ent_iob_
                        if tok.ent_iob_ != "O":
                            label += '-' + tok.ent_type_
                            #comment for adding new line after sentences
                            f.write(f'{tok} {label}\n')
                            print(tok,label)
                        else:
                            f.write(f'{tok} {"O"}\n')
                            #print(tok,"O")
                            jeton=str(tok)
                            if (jeton.__contains__(".")):
                                f.write('\n')
                                #print("separator propozitie")
                            else:
                                humpty_dumpty=2
                            next_step=0
                            #print("new line end of sentence2","   ")

            else:
                 next_step=0
            #doc=[]
            
                
        except:
            x=1
            #print("An exception occurred")
#         f.write('\n')
#         print("new line end of sentence3","   ")
                #print(tok, label, sep="\t")
# Add new line aftre sencences                
#                 if str(tok) == ".":
#                     f.write('\n')
#                 else:
#                     #f.write
#                     f.write(f'{tok} {label}\n')
#                     #print(i)

    

Same ideea like above but for case labels and spans. 

    file :conll_corpus_train_lab_patient_case.conll'

In [ ]:
data = TRAIN_DATA4

nlp = spacy.blank("en")

# Construction from class
# from spacy.pipeline import EntityRuler
# ruler = EntityRuler(nlp, overwrite_ents=True)
#myfile = open('xyz.txt', 'w')
i=0
with open('conll_corpus_train_lab_patient_case.conll', 'w') as f:
   
    for text, labels in data:
        doc=[]
        #print(text)
        doc = nlp(text)
    
        ents = []
        labels["entities"]=sorted(labels["entities"])
        for start, end, label in labels["entities"]:
            print(start,end,label)
           
            ents.append(doc.char_span(start, end, label))
        
        print(ents)
        try:
            doc.ents = ents
            next_step=0
            for tok in doc:
                label = tok.ent_iob_
                if tok.ent_iob_ != "O":
                    label += '-' + tok.ent_type_
                    #comment for adding new line after sentences
                    f.write(f'{tok} {label}\n')
                    next_step=1
                    f.write('\n')
                else:
                    humpty_dumpty=1
                if next_step==1:
                    for tok in doc:
                        label = tok.ent_iob_
                        if tok.ent_iob_ != "O":
                            label += '-' + tok.ent_type_
                            #comment for adding new line after sentences
                            f.write(f'{tok} {label}\n')
                        else:
                            f.write(f'{tok} {"O"}\n')
                    next_ste==0
                else:
                    next_step=0
            doc=[]
            f.write('\n')      
                #print(tok, label, sep="\t")
# Add new line aftre sencences                
#                 if str(tok) == ".":
#                     f.write('\n')
#                 else:
#                     #f.write
#                     f.write(f'{tok} {label}\n')
#                     #print(i)
        
        except:
            x=1

In [ ]:
#TRAIN_DATA4

## Consolidate Train corpus

In [ ]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head(10))

In [ ]:
train.case_num.value_counts()

In [ ]:
train['pn_history'][10]

In [ ]:
# incorrect annotation
train.loc[338, 'annotation'] = ast.literal_eval('[["father heart attack"]]')
train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

train.loc[621, 'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

train.loc[655, 'annotation'] = ast.literal_eval('[["no heat intolerance"], ["no cold intolerance"]]')
train.loc[655, 'location'] = ast.literal_eval('[["285 292;301 312"], ["285 287;296 312"]]')

train.loc[1262, 'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

train.loc[1265, 'annotation'] = ast.literal_eval('[[\'felt like he was going to "pass out"\']]')
train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

train.loc[1396, 'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

train.loc[1591, 'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

train.loc[1615, 'annotation'] = ast.literal_eval('[["diarrhea for last 2-3 days"]]')
train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

train.loc[1664, 'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

train.loc[1714, 'annotation'] = ast.literal_eval('[["started about 8-10 hours ago"]]')
train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

train.loc[1929, 'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

train.loc[2134, 'annotation'] = ast.literal_eval('[["last sexually active 9 months ago"]]')
train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

train.loc[2191, 'annotation'] = ast.literal_eval('[["right lower quadrant pain"]]')
train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

train.loc[2553, 'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

train.loc[3858, 'annotation'] = ast.literal_eval('[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]')
train.loc[3858, 'location'] = ast.literal_eval('[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]')

train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

train.loc[4782, 'annotation'] = ast.literal_eval('[["darker brown stools"]]')
train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

train.loc[4908, 'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

train.loc[6016, 'annotation'] = ast.literal_eval('[["difficulty falling asleep"]]')
train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

train.loc[6192, 'annotation'] = ast.literal_eval('[["helps to take care of aging mother and in-laws"]]')
train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

train.loc[6380, 'annotation'] = ast.literal_eval('[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]')
train.loc[6380, 'location'] = ast.literal_eval('[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]')

train.loc[6562, 'annotation'] = ast.literal_eval('[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]')
train.loc[6562, 'location'] = ast.literal_eval('[["290 320;327 337"], ["290 320;342 358"]]')

train.loc[6862, 'annotation'] = ast.literal_eval('[["stressor taking care of many sick family members"]]')
train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

train.loc[7022, 'annotation'] = ast.literal_eval('[["heart started racing and felt numbness for the 1st time in her finger tips"]]')
train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

train.loc[7422, 'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

train.loc[8876, 'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

train.loc[9027, 'annotation'] = ast.literal_eval('[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

train.loc[9938, 'annotation'] = ast.literal_eval('[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]')
train.loc[9938, 'location'] = ast.literal_eval('[["89 117"], ["122 138"], ["368 402"]]')

train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

train.loc[10513, 'annotation'] = ast.literal_eval('[["weight gain"], ["gain of 10-16lbs"]]')
train.loc[10513, 'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

train.loc[11551, 'annotation'] = ast.literal_eval('[["seeing her son knows are not real"]]')
train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

train.loc[11677, 'annotation'] = ast.literal_eval('[["saw him once in the kitchen after he died"]]')
train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

train.loc[12124, 'annotation'] = ast.literal_eval('[["tried Ambien but it didnt work"]]')
train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

train.loc[12279, 'annotation'] = ast.literal_eval('[["heard what she described as a party later than evening these things did not actually happen"]]')
train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

train.loc[12289, 'annotation'] = ast.literal_eval('[["experienced seeing her son at the kitchen table these things did not actually happen"]]')
train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

train.loc[13238, 'annotation'] = ast.literal_eval('[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
train.loc[13238, 'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

train.loc[13297, 'annotation'] = ast.literal_eval('[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]')
train.loc[13297, 'location'] = ast.literal_eval('[["182 221"], ["182 213;225 234"]]')

train.loc[13299, 'annotation'] = ast.literal_eval('[["yesterday"], ["yesterday"]]')
train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

train.loc[13845, 'annotation'] = ast.literal_eval('[["headache global"], ["headache throughout her head"]]')
train.loc[13845, 'location'] = ast.literal_eval('[["86 94;230 236"], ["86 94;237 256"]]')

train.loc[14083, 'annotation'] = ast.literal_eval('[["headache generalized in her head"]]')
train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')

In [ ]:
train.head(100)

In [ ]:
train['annotation_length'] = train['annotation'].apply(len)
display(train['annotation_length'].value_counts())

In [ ]:
train=train[train.annotation_length !=0]

In [ ]:
train=train.reset_index()

In [ ]:
import re
def extract_numbers(x):
        sentence = x
        s = [float(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
        return s

In [ ]:
train['location_string']=train['location'].apply(lambda x: str(x))

In [ ]:
 train.location_string[6]

In [ ]:
import re
sentence = train.location_string[5]
s = [float(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
print(s)

In [ ]:
train['spans']=train['location_string'].apply(lambda x: extract_numbers(x))

In [ ]:
display(train['annotation_length'].value_counts()) 

In [ ]:
train['span_length'] = train['spans'].apply(lambda x: len(x))
display(train['span_length'].value_counts())

In [ ]:
len(train)

In [ ]:
train['spans']=train['spans'].apply(lambda x: np.array(x, int))

In [ ]:
import re
def loc_coordinates(location):
    sentence = str(location)
    s = [int(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    return s

In [ ]:
train['span_length'] = train['spans'].apply(len)
display(train['span_length'].value_counts())

### Set labels

In [ ]:
train.head(100)

In [ ]:
train["labels"]=train["feature_num"].apply(lambda x:"NBME"+str(x))
LABEL=train["labels"].unique().tolist()

In [ ]:
train_2=train

In [ ]:
train_2.feature_text=train_2.feature_text.str.replace('-',' ')

In [ ]:
train_2['feature_text_lenght']=train_2['feature_text'].apply(lambda x: len(x))

In [ ]:
train_2['feature_text_span']=train_2['feature_text_lenght'].apply(lambda x: [0,x])

In [ ]:
train_2.head()

Spacy incorrect format

In [ ]:
# test=test.reset_index()

new tagger

In [ ]:
#temp_df=train[train.pn_num ==16]

In [ ]:
#len(temp_df)

In [ ]:
# def getOverlap(a, b):
#     return max(0, min(a[1], b[1]) - max(a[0], b[0]))
# span_list=[]
# for i in range(len(temp_df)):
#         k=0
#         l=0
#         if temp_df.span_length[i]==0 :
#                 print(1)
#                 span_list.append([0,0,i,k])
#         else:
#             rangex=(temp_df.span_length[i])/2
#             for j in range(int(rangex)):
#                 print("j:",j)
#                 if np.isnan(temp_df.span_length[i]) :
#                     print(2)
#                     span_list.append([0,0,i,k])
#                     print(k)
#                     #k=k+2
#                 else:
#                     print("j:",j)
                
#                     a=temp_df.spans[j]
#                     print(type(a))
#                     #b=temp_df.spans[k+1]
#                     #print(temp_df.span_length[i])
#                     print(a)
#                     print(a[k])
#                     print(a[k+1])
#                     #b=temp_df.spans[k+1]
#                     span_list.append([a,b,i,k])
#                     print(k)
#             k=k+2

In [ ]:
train_df=train

In [ ]:
TRAIN_DATA = []
tuple_list=[]
dict_list=[]
alpha=0
#for i in range(20):
for i in range(len(train_df)):
    
    #contor_lista=working_corpus.annotation_length[i]
    
    k=0
    #print(working_corpus.annotation_length[i])
    #print(location_is)
    dict_list=[]
    lista1=[]
    lista2=[]
    lista3=[]
    lista4=[]
    #print(train_df.span_length[i])
    alpha=int((train_df.span_length[i])/2)
    #print(alpha)
    #location_is=loc_coordinates(train_df["spans"][i])
    for j in range(alpha):
        text=train_df.pn_history[i]
#         if np.isnan(train_df["spans"][i])[1]:
#             print('no good')
#         else:
#             print('we are good')
        #print(location_is)
        #print(train_df["spans"][i])
        #print(location_is[k])
        spans=train_df["spans"][i]
        
        #print('spansx',spans[k])
        #print('spansy',spans[k+1])
        #print("lista este",lista1)
        #stringul=[]
                #stringul=[]
        #print(train_df["feature_num"][i])
        code_is="NBME"+str(train_df["feature_num"][i])
        dict_list.append([spans[k],spans[k+1],code_is])
        lista1=[spans[k],spans[k+1],code_is]
        lista2.append([spans[k],spans[k+1],code_is])
        #print(lista1)
        ent_dic = {}
    #     #ent_dic['entities'] = dict_list[-1]
        ent_dic['entities'] = [lista1] #single value lista2 multiple values 
    #     #tuple_list.append(tupla)
        TRAIN_DATA.append([text,ent_dic])
        #print(k)
#         ent_dic2={}
#         ent_dic2["entities"] = lista1
#         #tupla=(text,{"entities"+":", location_is[k],location_is[k+1]})
#         TRAIN_DATA.append([text,ent_dic2])
        k=k+2


In [ ]:
len(TRAIN_DATA)

In [ ]:

#TRAIN_DATA

CONLL format converter

In [ ]:
import sys

In [ ]:
# text=('control text .de . control. . si mai amabil')
# doc = nlp(text)
# doc[0]
# for tok in doc:
#     print(tok)
#     if str(tok) == ".":
#         print()
#         write('\n')

In [ ]:
len(TRAIN_DATA)

In [ ]:
#import spacy

data = TRAIN_DATA

nlp = spacy.blank("en")
#myfile = open('xyz.txt', 'w')
i=0
with open('conll_corpus_train_lab1x.conll', 'w') as f:
    doc=[]
    for text, labels in data:
        doc = nlp(text)
        ents = []
        for start, end, label in labels["entities"]:
            ents.append(doc.char_span(start, end, label))
        
        try:
            doc.ents = ents
            next_step=0
            for tok in doc:
                label = tok.ent_iob_
                if tok.ent_iob_ != "O":
                    label += '-' + tok.ent_type_
                    #comment for adding new line after sentences
                    f.write(f'{tok} {label}\n')
                    next_step=1
                    f.write('\n')
                else:
                    humpty_dumpty=1
                if next_step==1:
                    for tok in doc:
                        label = tok.ent_iob_
                        if tok.ent_iob_ != "O":
                            label += '-' + tok.ent_type_
                            #comment for adding new line after sentences
                            f.write(f'{tok} {label}\n')
                        else:
                            f.write(f'{tok} {"O"}\n')
                    next_ste==0
                else:
                    next_step=0
            f.write('\n')      
                #print(tok, label, sep="\t")
# Add new line aftre sencences                
#                 if str(tok) == ".":
#                     f.write('\n')
#                 else:
#                     #f.write
#                     f.write(f'{tok} {label}\n')
#                     #print(i)
        
        except:
            x=1
            #print("An exception occurred")
    

Labels by cases

In [ ]:
train["labels"]=train["case_num"].apply(lambda x:"NBME-Cno-"+str(x))
LABEL=train["labels"].unique().tolist()

In [ ]:
TRAIN_DATA = []
tuple_list=[]
dict_list=[]
alpha=0
#for i in range(20):
for i in range(len(train_df)):
    
    #contor_lista=working_corpus.annotation_length[i]
    
    k=0
    #print(working_corpus.annotation_length[i])
    #print(location_is)
    dict_list=[]
    lista1=[]
    lista2=[]
    lista3=[]
    lista4=[]
    alpha=int(train_df.span_length[i]/2)
    #print(alpha)
    for j in range(alpha):
        text=train_df.pn_history[i]
        location_is=loc_coordinates(train_df["spans"][i])
        
        
        #print("lista este",lista1)
        #stringul=[]
                #stringul=[]
        code_is="NBME-Cno-"+str(train_df["case_num"][i])
        dict_list.append([location_is[k],location_is[k+1],code_is])
        lista1=[location_is[k],location_is[k+1],code_is]
        lista2.append([location_is[k],location_is[k+1],code_is])
        ent_dic = {}
    #     #ent_dic['entities'] = dict_list[-1]
        ent_dic['entities'] = [lista1] #single value lista2 multiple values 
    #     #tuple_list.append(tupla)
        TRAIN_DATA.append([text,ent_dic])
        #print(k)
#         ent_dic2={}
#         ent_dic2["entities"] = lista1
#         #tupla=(text,{"entities"+":", location_is[k],location_is[k+1]})
#         TRAIN_DATA.append([text,ent_dic2])
        k=k+2


In [ ]:
len(TRAIN_DATA)

In [ ]:
TRAIN_DATA2 = []
tuple_list=[]
dict_list=[]
alpha=0
#for i in range(20):
for i in range(len(train_2)):
    
    #contor_lista=working_corpus.annotation_length[i]
    
    k=0
    #print(working_corpus.annotation_length[i])
    #print(location_is)
    dict_list=[]
    lista1=[]
    lista2=[]
    lista3=[]
    lista4=[]
    #print(train_df.span_length[i])
    #alpha=int((train_2.span_length[i])/2)
    #print(alpha)
    #location_is=loc_coordinates(train_df["spans"][i])

    text=train_2.feature_text[i]
#         if np.isnan(train_df["spans"][i])[1]:
#             print('no good')
#         else:
#             print('we are good')
    #print(location_is)
    #print(train_2["spans"][i])
    #print(location_is[k])
    spans=train_2["feature_text_span"][i]

    #print('spansx',spans[k])
    #print('spansy',spans[k+1])
    #print("lista este",lista1)
    #stringul=[]
            #stringul=[]
    #print(train_df["feature_num"][i])
    code_is="NBME"+str(train_2["feature_num"][i])
    dict_list.append([spans[0],spans[1],code_is])
    lista1=[spans[0],spans[1],code_is]
    lista2.append([spans[0],spans[1],code_is])
    #print(lista1)
    ent_dic = {}
#     #ent_dic['entities'] = dict_list[-1]
    ent_dic['entities'] = [lista1] #single value lista2 multiple values 
#     #tuple_list.append(tupla)
    TRAIN_DATA2.append([text,ent_dic])
    #print(k)
#         ent_dic2={}
#         ent_dic2["entities"] = lista1
#         #tupla=(text,{"entities"+":", location_is[k],location_is[k+1]})
#         TRAIN_DATA.append([text,ent_dic2])


In [ ]:
data = TRAIN_DATA2

nlp = spacy.blank("en")
#myfile = open('xyz.txt', 'w')
#f=open("guru99.txt", "a+")
i=0
with open('conll_corpus_train_lab1y.conll', 'a+') as f:
    doc=[]
    for text, labels in data:
        doc = nlp(text)
        ents = []
        for start, end, label in labels["entities"]:
            ents.append(doc.char_span(start, end, label))
        
        try:
            doc.ents = ents
            next_step=0
            for tok in doc:
                label = tok.ent_iob_
                if tok.ent_iob_ != "O":
                    label += '-' + tok.ent_type_
                    #comment for adding new line after sentences
                    f.write(f'{tok} {label}\n')
                    next_step=1
                    f.write('\n')
                if next_step==1:
                
                    for tok in doc:
                        label = tok.ent_iob_
                        if tok.ent_iob_ != "O":
                            label += '-' + tok.ent_type_
                            #comment for adding new line after sentences
                            f.write(f'{tok} {label}\n')
                        else:
                            f.write(f'{tok} {"O"}\n')
                else:
                    next_step=0
            f.write('\n')      
# Add new line aftre sencences                
#                 if str(tok) == ".":
#                     f.write('\n')
#                 else:
#                     #f.write
#                     f.write(f'{tok} {label}\n')
#                     #print(i)
        
        except:
            x=1
            #print("An exception occurred")
    

In [ ]:
data = TRAIN_DATA

nlp = spacy.blank("en")
#myfile = open('xyz.txt', 'w')
i=0
with open('conll_corpus_train_lab2', 'w') as f:

    for text, labels in data:
        doc = nlp(text)
        ents = []
        for start, end, label in labels["entities"]:
            ents.append(doc.char_span(start, end, label))
        
        try:
            doc.ents = ents
            for tok in doc:
                label = tok.ent_iob_
                if tok.ent_iob_ != "O":
                    label += '-' + tok.ent_type_
                    #comment for adding new line after sentences
                    f.write(f'{tok} {label}\n')
                #print(tok, label, sep="\t")
# Add new line aftre sencences                
#                 if str(tok) == ".":
#                     f.write('\n')
#                 else:
#                     #f.write
#                     f.write(f'{tok} {label}\n')
#                     #print(i)
        except:
            x=1
            #print("An exception occurred")
    f.write('\n')